In [1]:
import os
import monai
import torch
from monai.data import DataLoader, Dataset, load_decathlon_datalist, NibabelWriter
from monai.data.utils import no_collation
from monai.transforms import (
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    LoadImaged,
    Orientationd,
    Spacingd,
)

In [2]:
monai.config.print_config()

MONAI version: 1.2.0
Numpy version: 1.24.4
Pytorch version: 2.0.1+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /usr/local/lib/python3.8/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.21.0
Pillow version: 10.0.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2+cu118
tqdm version: 4.66.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.0.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 2.6.0
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommend

In [3]:
spacing = [0.703125, 0.703125, 1.25]
data_base_dir = '/data/HC_Images_resample/'
data_list_file_path = '/data/output/hc_train_val.json'
# data_list_file_path = '/data/output/hc_test.json'
orig_data_base_dir = '/data/HC1/'
# data_base_dir = '/data/MSD_Images_resample/'
# data_list_file_path = '/data/output/msd_train_val.json'
# data_list_file_path = '/data/output/msd_test.json'
# orig_data_base_dir = '/data/MSD-files-moved/'

In [4]:
process_transforms = Compose(
    [
        LoadImaged(
            keys=["image"],
            meta_key_postfix="meta_dict",
            reader="itkreader",
            affine_lps_to_ras=True
        ),
        EnsureChannelFirstd(keys=["image"]),
        EnsureTyped(keys=["image"], dtype=torch.float16),
        Orientationd(keys=["image"], axcodes="RAS"),
        Spacingd(keys=["image"], pixdim=spacing, padding_mode="border"),
    ]
)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


In [5]:
for data_list_key in ["training", "validation"]:
# for data_list_key in ["test"]:
    process_data = load_decathlon_datalist(
        data_list_file_path,
        is_segmentation=True,
        data_list_key=data_list_key,
        base_dir=orig_data_base_dir
    )
    process_ds = Dataset(
        data=process_data,
        transform=process_transforms
    )
    process_loader = DataLoader(
        process_ds,
        batch_size=1,
        shuffle=False,
        pin_memory=False,
        collate_fn=no_collation
    )

    print("-" * 10)
    for batch_data in process_loader:
        for batch_data_i in batch_data:
            subj_id = batch_data_i["image_meta_dict"]["filename_or_obj"].split("/")[3] + '.nii.gz'
            # subj_id = batch_data_i["image_meta_dict"]["filename_or_obj"].split("/")[-1]
            new_filename = os.path.join(data_base_dir, subj_id)
            writer = NibabelWriter()
            writer.set_data_array(data_array=batch_data_i["image"])
            writer.set_metadata(meta_dict=batch_data_i["image"].meta)
            writer.write(new_filename, verbose=True)

----------
2023-08-26 19:10:40,507 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL652474346066949.nii.gz
2023-08-26 19:11:24,428 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL883384319849708.nii.gz
2023-08-26 19:12:04,234 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL392606925066512.nii.gz
2023-08-26 19:12:37,808 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL176076668767205.nii.gz
2023-08-26 19:13:18,801 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL140377731469643.nii.gz
2023-08-26 19:14:05,406 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL372317261298557.nii.gz
2023-08-26 19:14:45,969 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL983603910175928.nii.gz
2023-08-26 19:15:28,790 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL210874587126061.nii.gz
2023-08-26 19:16:17,085 INFO image_writer.py:197 - writing: /data/HC_Images_resample/PL455075220933606.nii.gz